<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/Archive_org_Redirect_Mapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install urlextract
!pip install polyfuzz
!pip install waybackpy
!pip install bs4
!pip install imgaug==0.2.5
!pip install folium==0.2.1

     |████████████████████████████████| 15.7MB 181kB/s 
     |████████████████████████████████| 757kB 23.0MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


     |████████████████████████████████| 563kB 7.7MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp37-none-any.whl size=561439 sha256=28e1fc3001a7bc344aa5a191177df06a01ef8fc4b50bae45ede05e7ac3baf9cc
  Stored in directory: /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
     |████████████████████████████████| 71kB 4.7MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp37-none-any.whl size=79810 sha256=96990d188df2da66cb0dff2725da9ed5763823e7441fa7891f6945cff829e3cf
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
from io import StringIO
import io
from urllib.parse import urlparse
from urllib.request import urlopen

import pandas as pd
import requests as req
from bs4 import BeautifulSoup
from polyfuzz import PolyFuzz
from waybackpy import Cdx
from google.colab import files

In [ ]:
# set the user agent here
user_agent = "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"

In [ ]:
# upload the Screaming Frog crawl File (internal_html.csv)
uploaded = files.upload()

TypeError: ignored

In [ ]:
df_sf = pd.read_csv(io.StringIO(uploaded['internal_html.csv'].decode('utf-8')))
print("Imported Crawl File ..")

In [ ]:
# extract the domain name from the crawl
extracted_domain = df_sf["Address"]
extracted_domain = extracted_domain.iloc[0]
url = extracted_domain
o = urlparse(url)
domain = o.netloc
print("Detected Domain = ", domain)

In [ ]:
# append extracted domain to archive.org txt url
archive_url = "http://web.archive.org/cdx/search/cdx?url=" + domain + "*&output=txt"

In [ ]:
# requests - set the session user agent
session = req.Session()
session.headers.update({'User-Agent': user_agent})

In [ ]:
# get the response
print("\nDownloading URLs from Archive.org ..")
resp = session.get(archive_url)

In [ ]:
# make the df and set the column names
df_archive = pd.read_csv(
    StringIO(resp.text),
    sep=" ",
    names=["1", "2", "Address", "Content Type", "5", "6", "7"],
    dtype=str,
)

In [ ]:
# count the rows
count_row = df_archive.shape[0]
print("\nDownloaded", count_row, "URLs from Archive.org ..")

In [ ]:
# replace port 80 urls before de-duping
df_archive["Address"] = df_archive["Address"].str.replace("\:80", "")

In [ ]:
# drop duplicate urls
df_archive.drop_duplicates(subset="Address", inplace=True)

In [ ]:
# keep only text/http content
df_archive = df_archive[df_archive["Content Type"].isin(["text/html"])]
df_archive["Address"].str.lower()

In [ ]:
# drop additional non-html pages incorrectly flagged as text/html
df_archive = df_archive[
    ~df_archive["Address"].str.contains(
        ".css|.js|.jpg|.png|.jpeg|.pdf|.JPG|.PNG|.CSS|.JS|.JPEG|.PDF|.ICO|.GIF|.TXT|.ico|ver=|.gif|.txt")]

In [ ]:
# drop marketing tags
df_archive = df_archive[~df_archive["Address"].str.contains("utm|gclid")]

In [ ]:
# drop any additional rows (custom) (Drops brand pages and pagination)
df_archive = df_archive[~df_archive["Address"].str.contains("/brand|/page|/p=")]

In [ ]:
# drop basket related URLs [Should be blocked in robots.txt anyway!]
df_archive = df_archive[~df_archive["Address"].str.contains("basket|checkout|add|account")]

In [ ]:
# checks if archive.org url is found in the crawl (and drop it if it is)
df_archive["Match"] = df_archive["Address"].str.contains('|'.join(df_sf['Address']), case=False)

In [ ]:
# drop matches
df_archive = df_archive[~df_archive["Match"].isin([True])]

In [ ]:
# reindex the columns
df_archive = df_archive.reindex(columns=['Address'])

In [ ]:
# count the remaining rows
remaining_count = df_archive.shape[0]
print("\nFiltered to", remaining_count, "Qualifying URLs!")

In [ ]:
# # extract valid URLs from list
# print("Checking Validity of URLs and removing any which are malformed ..")
# extractor = URLExtract()
# df_archive['Extracted URL'] = df_archive['Address'].apply(extractor.find_urls)
# print(df_archive['Extracted URL'])

In [ ]:
df_archive = df_archive[df_archive["Address"].notna()]

In [ ]:
# drop port 80 urls
df_archive = df_archive[~df_archive["Address"].str.contains(":80", na=False)]

In [ ]:
# create list from address column
url_list = df_archive['Address']

In [ ]:
# empty list to append to
archive_url_list = []

In [ ]:
print("\nGetting URLs from Archive.org to scrape H1s ..")
for url in url_list:
  try:
      cdx = Cdx(url=url, user_agent=user_agent)
      snapshots = cdx.snapshots()

      for snapshot in snapshots:
          print(snapshot.archive_url)
          archive_url_list.append(snapshot.archive_url)
  except ValueError:
    print("Value Error!")
    pass

In [ ]:
df_temp = pd.DataFrame(archive_url_list, columns=['Recovered Archive URL'])

In [ ]:
# extract original url from Recovered Archive URL (for de-duping) and clean the data
df_temp["Original URL"] = (df_temp["Recovered Archive URL"].str.split("/").str[5:])
df_temp["Original URL"] = df_temp["Original URL"].str.join(",")
df_temp["Original URL"] = df_temp["Original URL"].str.replace(',', '/')

In [ ]:
# drop port 80 urls
df_temp = df_temp[~df_temp["Original URL"].str.contains(":80", na=False)]

In [ ]:
# sort by oldest urls
df_temp = df_temp.sort_values(by="Recovered Archive URL", ascending=True)

In [ ]:
# drop duplicates
df_temp.drop_duplicates(subset=['Original URL'], keep="first", inplace=True)

In [ ]:
# extract h1s from temp df and make into a list
archive_url_list = df_temp['Recovered Archive URL']

In [ ]:
print("\nScraping H1s from Archive.org\n")

In [ ]:
archive_h1_list = []
for i in archive_url_list:
    try:
        html = urlopen(i)
        bsh = BeautifulSoup(html.read(), 'html.parser')
        print(bsh.h1.text.strip())
        archive_h1_list.append(bsh.h1.text.strip())
    except AttributeError:
        print("Got an HTTP 301 response at crawl time")
        archive_h1_list.append("Got an HTTP 301 response at crawl time")
    except Exception:
        print("Error getting URL")
        archive_h1_list.append("Error getting URL")
        pass

In [ ]:
# add list to dataframe column
df_temp['H1'] = archive_h1_list

In [ ]:
# drop redirected urls
df_temp = df_temp[~df_temp["H1"].isin(["Got an HTTP 301 response at crawl time"])]

In [ ]:
# merge back into main dataframe
df_archive = pd.merge(df_archive, df_temp, left_on="Address", right_on="Original URL", how="inner")

In [ ]:
# start poly fuzz
df_archive = df_archive[df_archive["H1"].notna()]
df_sf = df_sf[df_sf["H1-1"].notna()]

In [ ]:
# create lists from dfs
df_sf_list = list(df_sf["H1-1"])
df_archive_list = list(df_archive["H1"])

In [ ]:
# instantiate PolyFuzz model, choose TF-IDF as the similarity measure and match the two lists.
model = PolyFuzz("TF-IDF").match(df_archive_list, df_sf_list)

In [ ]:
# make the polyfuzz dataframe
df_matches = model.get_matches()

In [ ]:
# make mini dataframe
df_sf_mini = df_sf[['H1-1', 'Address']]

In [ ]:
# merge back in
df_archive = pd.merge(df_archive, df_matches, left_on="H1", right_on="From", how="inner")
df_archive = pd.merge(df_archive, df_sf_mini, left_on="To", right_on="H1-1")

In [ ]:
# drop duplicate rows
df_archive.drop_duplicates(subset=['Address_x'], keep="first", inplace=True)

In [ ]:
# reindex the columns
df_archive = df_archive.reindex(columns=['Address_x', 'Address_y', 'Similarity'])

In [ ]:
# rename the columns
df_archive.rename(columns={'Address_x': 'Recovered Archive URL', "Address_y": "Live URL"}, inplace=True)

In [ ]:
# count the final datframe rows
count_row = df_archive.shape[0]

In [ ]:
# check http status of recovered archive.org url
# make the list
recovered_archive_list = df_archive['Recovered Archive URL']

In [ ]:
print("\nGetting HTTP Status of Source URL..\n")

In [ ]:
count = 0
recovered_status_list = []
for url in recovered_archive_list:
    try:
        url = session.head(i).status_code
        count = count + 1
        print("Crawled", count, "of", count_row, "URLs", "Status:", url)
        recovered_status_list.append(url)
    except Exception:
        print("Exception!")
        recovered_status_list.append("Exception Error")
        pass

In [ ]:
df_archive['Status Code'] = recovered_status_list

In [ ]:
# Change the Datatype for Filtering
df_archive['Status Code'] = df_archive['Status Code'].astype(str)

In [ ]:
# Filter Out Non-Valid Stats Codes (E.g. Already Redirected)
df_archive = df_archive[~df_archive['Status Code'].str.contains("200|301|302|500|503")]

In [ ]:
# get the final count
final_count = df_archive.shape[0]

In [ ]:
print("\nTotal Valid Opportunity", final_count, "URLS")

In [ ]:
df_archive.to_csv('urls-to-redirect-archive-org.csv')
files.download('urls-to-redirect-archive-org.csv')